Topics to Cover in today's workshop:

1. Sentiment Classification
2. Fake News DEtection
    a. MLP <br>
    b. Feed Forward NN <br>
    c. LSTM <br>
    d. CNN <br>
    e. BERT <br>
    f. BERT + Fine Tuning <br>
3. Word Embeddings
    a. Word2Vec <br>
    b. Glove<br>
    c. BERT Embeddings<br>

In [74]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

Dataset

In [75]:
df = pd.read_csv('/content/drive/MyDrive/IIITD Data/NLP/workshop/data.tsv', sep='\t')
df.head()

,Phrase,Sentiment
0,"The gags , and the script , are a mixed bag .",0
1,is made fresh by an intelligent screenplay and...,1
2,would n't matter so much that this arrogant Ri...,0
3,to ask people to sit still for two hours and c...,0
4,The story gives ample opportunity for large-sc...,1


Preprocessing 

In [77]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [78]:
# Combining all the above stundents 
from tqdm import tqdm
from bs4 import BeautifulSoup

preprocessed_reviews = []

for sentance in tqdm(df['Phrase'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split())
    #uncomment below if you want to remove stopwords too
    # sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

# new column feature for proecssed reviews
df['reviews'] = preprocessed_reviews
df.head()

100%|██████████| 12000/12000 [00:03<00:00, 3822.36it/s]


,Phrase,Sentiment,reviews
0,"The gags , and the script , are a mixed bag .",0,the gags and the script are a mixed bag
1,is made fresh by an intelligent screenplay and...,1,is made fresh by an intelligent screenplay and...
2,would n't matter so much that this arrogant Ri...,0,would not matter so much that this arrogant ri...
3,to ask people to sit still for two hours and c...,0,to ask people to sit still for two hours and c...
4,The story gives ample opportunity for large-sc...,1,the story gives ample opportunity for large sc...


In [79]:
from sklearn.model_selection import train_test_split

# Data Splitting
# Train Data :  80%
# Test Data  :  20%

x = df['reviews']
y = np.array(df['Sentiment'])

# data split  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)
print(f"Train Data Size: {x_train.shape[0]}")
print(f"Test Data Size: {x_test.shape[0]}")

Train Data Size: 9600
Test Data Size: 2400


# Word Embeddings

1. Word2Vec Embeddings

In [81]:
# specify names for word2vec data
X_train_w2v, X_test_w2v = list(x_train), list(x_test)
y_train_w2v, y_test_w2v = y_train, y_test

# create 
i=0
list_of_sentance_test=[]
for sentance in X_test_w2v:
    list_of_sentance_test.append(sentance.split())

i=0
list_of_sentance=[]
for sentance in X_train_w2v:
    list_of_sentance.append(sentance.split())

In [82]:
# Using Google News Word2Vectors

# min_count = 5 considers only words that occured atleast 5 times
w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
print(w2v_model.wv.most_similar('great'))
print('='*50)
print(w2v_model.wv.most_similar('worst'))

[('star', 0.9996079802513123), ('dull', 0.9996078610420227), ('day', 0.9995817542076111), ('compelling', 0.9995776414871216), ('itself', 0.9995694160461426), ('girl', 0.9995529651641846), ('comes', 0.9995518922805786), ('fascinating', 0.9995502233505249), ('work', 0.9995487928390503), ('takes', 0.9995459318161011)]
[('summer', 0.9992620348930359), ('films', 0.9992106556892395), ('rare', 0.9992033839225769), ('work', 0.9991836547851562), ('man', 0.9991766810417175), ('stories', 0.9991741180419922), ('great', 0.9991329312324524), ('picture', 0.9991177916526794), ('version', 0.9991052150726318), ('making', 0.9990689754486084)]


In [83]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  4185
sample words  ['the', 'film', 'does', 'not', 'have', 'enough', 'innovation', 'or', 'to', 'attract', 'teenagers', 'and', 'it', 'lacks', 'novel', 'charm', 'that', 'made', 'spy', 'kids', 'a', 'surprising', 'winner', 'with', 'both', 'adults', 'younger', 'audiences', 'maybe', 'how', 'will', 'you', 'feel', 'after', 'an', 'rip', 'off', 'of', 'rock', 'action', 'slo', 'mo', 'gun', 'firing', 'random', 'glass', 'being', 'dreary', 'mid', 'hoffman']


In [84]:
# TRAINING W2V DATA VECTORS

# average Word2Vec
# compute average word2vec for each review.
sent_vectors_train = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) 
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_train.append(sent_vec)

print(len(sent_vectors_train))
print(len(sent_vectors_train[0]))

100%|██████████| 9600/9600 [00:03<00:00, 2618.64it/s]

9600
50


In [85]:
# TEST W2V DATA VECTORS

sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_test): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
    
print(len(sent_vectors_test))
print(len(sent_vectors_test[0]))

100%|██████████| 2400/2400 [00:00<00:00, 2547.39it/s]

2400
50


In [86]:
X_train_avgw2v, X_test_avgw2v = sent_vectors_train, sent_vectors_test

# MODELS

MLP

In [87]:
#relevant libraries
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch

In [88]:
#data into pytorch tensor
x_train_w2v = torch.tensor(X_train_avgw2v, dtype=torch.float32)
x_test_w2v = torch.tensor(X_test_avgw2v, dtype=torch.float32)

y_train_w2v = torch.from_numpy(y_train.astype(np.float32)) 
y_train_w2v = y_train_w2v.view(y_train_w2v.shape[0], 1)
y_test_w2v = torch.from_numpy(y_test.astype(np.float32)) 
y_test_w2v = y_test_w2v.view(y_test_w2v.shape[0], 1)

In [66]:
# do mlp here
clf = MLPClassifier(random_state=1, max_iter=300, activation='tanh', learning_rate='adaptive').fit(x_train_w2v, y_train_w2v)
y_pred = clf.predict(x_test_w2v)
print("Accuracy on Test Data", 100*accuracy_score(y_test_w2v, y_pred))

Accuracy on Test Data 57.49999999999999


Feed Forward Neural Network

In [89]:
# relevant libraries
import torch.nn as nn
import torch.nn.functional as F

In [90]:
#data into pytorch tensor
x_train_w2v = torch.tensor(X_train_avgw2v, dtype=torch.float32)
x_test_w2v = torch.tensor(X_test_avgw2v, dtype=torch.float32)

y_train_w2v = torch.from_numpy(y_train.astype(np.float32)) 
y_train_w2v = y_train_w2v.view(y_train_w2v.shape[0], 1)
y_test_w2v = torch.from_numpy(y_test.astype(np.float32)) 
y_test_w2v = y_test_w2v.view(y_test_w2v.shape[0], 1)

In [92]:
 class NeturalNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeturalNet, self).__init__()
        
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, 1)
        self.actv = nn.Sigmoid()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.actv(out)
        return out # see why we didn't put any softmax layer at the last?

In [93]:
# parameters
input_size = sent_vectors_train[0].shape[0]
hidden_size = 30
num_classes = len(np.unique(y_train_w2v))

# call our Neural Nework model here
model = NeturalNet(input_size, hidden_size, num_classes)

# loss and optimzer
# loss_def = nn.CrossEntropyLoss()
loss_def = nn.BCELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

# training loop
epochs = 2000
for epoch in range(epochs):

    # forward
    outputs = model(x_train_w2v)
    loss = loss_def(outputs, y_train_w2v)
  
    # backward
    optimiser.zero_grad()
    loss.backward() # dl/dw
    optimiser.step()

    # print some records
    if epoch%100 == 0:
        print(f"Completed for Epoch={epoch+100} , current loss= {loss.item()}")

Completed for Epoch=100 , current loss= 0.6962115168571472
Completed for Epoch=200 , current loss= 0.672905683517456
Completed for Epoch=300 , current loss= 0.6682748198509216
Completed for Epoch=400 , current loss= 0.6632984280586243
Completed for Epoch=500 , current loss= 0.6587013006210327
Completed for Epoch=600 , current loss= 0.656630277633667
Completed for Epoch=700 , current loss= 0.6530822515487671
Completed for Epoch=800 , current loss= 0.6512198448181152
Completed for Epoch=900 , current loss= 0.6499900221824646
Completed for Epoch=1000 , current loss= 0.6484162211418152
Completed for Epoch=1100 , current loss= 0.6547624468803406
Completed for Epoch=1200 , current loss= 0.6485295295715332
Completed for Epoch=1300 , current loss= 0.6454678177833557
Completed for Epoch=1400 , current loss= 0.6535665988922119
Completed for Epoch=1500 , current loss= 0.6439165472984314
Completed for Epoch=1600 , current loss= 0.6429005861282349
Completed for Epoch=1700 , current loss= 0.64376384

In [24]:
# for testing
with torch.no_grad():
    y_predicted_sig = model(x_test_w2v)
    y_pred = y_predicted_sig.round() # 0.9 -> 1
    acc = accuracy_score(y_test_w2v, y_pred)
print("Accuracy of Feed Forward Network", acc)

Accuracy of Feed Forward Network 0.6004166666666667
